In [160]:
import pandas as pd
import requests
from io import StringIO

In [161]:
response = requests.get('https://dl.dropbox.com/s/06ni77wvrhukth3/scimagojr_full_2020-03-31.csv')
content = StringIO(response.content.decode("utf8"))
df = pd.read_table(content, sep=";")
df['Quartile'] = df.apply(lambda row: 4 if row['SJR Quartile'] == '-' else int(row['SJR Quartile'][1]), axis = 1)

df.drop(['Unnamed: 0', 'Rank', 'Sourceid', 'Issn', 'SJR', 'SJR Quartile',
 'H index', 'Total Docs. (2018)', 'Total Docs. (3years)', 'Total Refs.',
 'Total Cites (3years)', 'Citable Docs. (3years)', 'Cites / Doc. (2years)',
 'Ref. / Doc.', 'Country', 'Publisher', 'Coverage', 'Categories'], axis=1, inplace=True)

In [162]:
print(df.columns.values)
df.sample(10)

['Title' 'Type' 'Area' 'Quartile']


,Title,Type,Area,Quartile
64171,Indian Journal of Sexually Transmitted Diseases,journal,"Public Health, Environmental and Occupational ...",4
49711,2015 IEEE Symposium on Communications and Vehi...,conference and proceedings,Mechanics of Materials,4
12735,Integrative Medicine,journal,Complementary and Alternative Medicine,3
55190,Amyotrophic Lateral Sclerosis and Frontotempor...,journal,Neurology,1
64760,Journal of Ultrasound,journal,"Radiology, Nuclear Medicine and Imaging",3
46619,International Journal of Foresight and Innovat...,journal,Management of Technology and Innovation,3
45431,Slavisticna Revija,journal,Literature and Literary Theory,1
9365,American Heart Journal,journal,Cardiology and Cardiovascular Medicine,1
31440,Revista em Agronegocio e Meio Ambiente,journal,Environmental Science (miscellaneous),4
44672,Communication Disorders Quarterly,journal,Linguistics and Language,2


In [163]:
# df_cs = df.loc[df['Area'].str.contains('computer science', case = False, regex = False) & df['SJR Quartile'].str.contains('Q1|Q2')]
df_cs = df.loc[df['Area'].str.contains('Computer Science', case = False, regex = False)]
df_cs['Area'] = 'Computer Science'
# df_ling = df.loc[df['Area'].str.contains('linguistics', case = False, regex = False) & (df['SJR Quartile'].str.contains('Q1|Q2'))]
df_ling = df.loc[df['Area'].str.contains('Linguistics', case = False, regex = False)]
df_ling['Area'] = 'Linguistics'

/home/jupyter_notebook/venv/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/jupyter_notebook/venv/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [164]:
ling_cs = pd.merge(df_cs, df_ling, how='inner', on=['Title', 'Type'],  suffixes=('_cs', '_ling'))
ling_cs = ling_cs.groupby(by=["Title", "Type"]).agg({'Quartile_ling': ['min', 'max'], 'Quartile_cs': ['min', 'max']})
ling_cs


Quartile_ling  \
                                                                                        min   
Title                                              Type                                       
2009 IEEE International Technology Management C... conference and proceedings             4   
2015 5th International Conference on Digital In... conference and proceedings             4   
2015 IEEE 7th International Workshop on Managin... conference and proceedings             4   
2015 IEEE Conference on Wireless Sensors, ICWiS... conference and proceedings             4   
2015 IRCOBI Conference Proceedings - Internatio... conference and proceedings             4   
2015 International Symposium on Intelligent Sig... conference and proceedings             4   
ACM International Conference Proceeding Series     conference and proceedings             4   
Applied Ontology                                   journal                                1   
Argument and Computation                           journal                                1   
CALL-EJ                                            journal                                2   
Computational Linguistics                          journal                                1   
Computational Linguistics in the Netherlands Jo... conference and proceedings             4   
Computer Assisted Language Learning                journal                                1   
Computers and Composition                          journal                                1   
Digital Scholarship in the Humanities              journal                                2   
ICCSE 2016 - 11th International Conference on C... conference and proceedings             4   
International Journal of Computer-Assisted Lang... journal                                2   
JALT CALL Journal                                  journal                                2   
Journal of Logic, Language and Information         journal                                2   
Journal of Visual Languages and Computing          journal                                2   
Komp'juternaja Lingvistika i Intellektual'nye T... conference and proceedings             4   
Language Documentation and Conservation            journal                                1   
Language Learning and Technology                   journal                                1   
Perspective Technologies and Methods in MEMS De... conference and proceedings             4   
Pragmatics and Cognition                           journal                                2   
Proceedings of 2016 8th International Conferenc... conference and proceedings             4   
Procesamiento de Lenguaje Natural                  journal                                2   
ReCALL                                             journal                                1   
Speech Communication                               journal                                1   
Synthesis Lectures on Human Language Technologies  book series                            1   
TAL Traitement Automatique des Langues             journal                                3   
Texto Livre                                        journal                                4   

                                                                                   \
                                                                              max   
Title                                              Type                             
2009 IEEE International Technology Management C... conference and proceedings   4   
2015 5th International Conference on Digital In... conference and proceedings   4   
2015 IEEE 7th International Workshop on Managin... conference and proceedings   4   
2015 IEEE Conference on Wireless Sensors, ICWiS... conference and proceedings   4   
2015 IRCOBI Conference Proceedings - Internatio... conference and proceedings   4   
2015 International Symposium on Intelligent Sig... conference and procee

In [165]:
s1.to_csv('res.csv', sep=";")